# Spam Detection

In this Notebook I will explore text message data downloaded from Kaggle and create models to predict if a message is spam or not. 

In [2]:
import pandas as pd
import numpy as np
import os

spam_file = os.path.join("Resources", "sms_spam.csv")
spam_data = pd.read_csv(spam_file)

# Recoding the type to 1-Spam and 0-Ham
spam_data['target'] = np.where(spam_data['type']=='spam',1,0)
spam_data.head(10)

,type,text,target
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [3]:
from sklearn.model_selection import train_test_split

# Split the data into Train and test sets
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Spam Ratio

*This function returns a float, the percent value (i.e. $ratio * 100$).

In [4]:
def spam_percent(): 
    result = 100*len(spam_data[spam_data['target']==1])/(1.0*len(spam_data))
    return result

spam_percent()

13.40150699677072

#### Longest Token

We fit the training data `X_train` using a Count Vectorizer with default parameters.

then we identify the longest token in the vocabulary.

*This function returns the longest token.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)
def long_token():
    return max(vect.get_feature_names(), key=lambda token:len(token))

long_token()

'hypotheticalhuagauahahuagahyuhagga'

### Naive Bayes classifier

We use the Count Vectorizer with default parameters created earlier.

then , we fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. And find the area under the curve (AUC) score using the transformed test data.

*The function below returns the AUC score as a float.*

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def nbc_auc():
    X_train_vectorized = vect.transform(X_train)
    clfrNB = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = clfrNB.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

nbc_auc()

0.9634978365302715

### using a Tfidf Vectorizer

We fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

Then we identify the 20 features with the smallest tf-idf and the 20 with the largest tf-idf?

we put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series is the feature name, and the data is the tf-idf.

The series of 20 features with smallest tf-idfs are sorted smallest tfidf first, the list of 20 features with largest tf-idfs are sorted largest first. 

*This function returns a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idfs_smalllarge():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    smallest = pd.Series(X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[:20]], index = feature_names[sorted_tfidf_index[:20]])
    largest = pd.Series(X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[:-21:-1]], index = feature_names[sorted_tfidf_index[:-21:-1]])
    
    return (smallest, largest)

tf_idfs_smalllarge()

(exterminator    0.074408
 organizer       0.074408
 pudunga         0.074408
 psychologist    0.074408
 psychiatrist    0.074408
 venaam          0.074408
 athletic        0.074408
 pest            0.074408
 chef            0.074408
 aaniye          0.074408
 companion       0.074408
 healer          0.074408
 stylist         0.074408
 dependable      0.074408
 sympathetic     0.074408
 determined      0.074408
 listener        0.074408
 courageous      0.074408
 driver          0.074408
 independence    0.091685
 dtype: float64, done         1.000000
 where        1.000000
 146tf150p    1.000000
 nite         1.000000
 645          1.000000
 ok           1.000000
 home         1.000000
 too          1.000000
 yup          1.000000
 alrite       1.000000
 okie         1.000000
 beerage      1.000000
 anytime      1.000000
 havent       1.000000
 thanx        1.000000
 lei          1.000000
 thank        1.000000
 tick         0.979385
 same         0.934766
 blank        0.931963
 dty

### Ignoring unfrequent terms

We fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function returns the AUC score as a float.*

In [8]:
def freqterm_nbc_auc():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clfrNB = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = clfrNB.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

freqterm_nbc_auc()

0.9506958877039964

### Length of documents

*The function below returns a tuple (average length not spam, average length spam).*

In [9]:
def avg_doc_length():
    
    
    nospam = np.mean(spam_data[spam_data['target']==0]['text'].str.len())
    spam = np.mean(spam_data[spam_data['target']==1]['text'].str.len())
    
    return (nospam, spam)

avg_doc_length()

(71.43981769214834, 138.63453815261045)

### Function to add new features
The following function combines new features into the training data:

In [10]:
def add_feature(X, feature_to_add):
#    """
#    Returns sparse feature matrix with added feature.
#    feature_to_add can also be a list of features.
#    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Adding the length of document

We fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, we fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function returns the AUC score as a float.*

In [11]:
from sklearn.svm import SVC

def docl_svc_auc():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_Train_add = add_feature(X_train_vectorized, X_train.str.len())
    X_test_vectorized = vect.transform(X_test)
    X_Test_add = add_feature(X_test_vectorized, X_test.str.len())
    clfrNB = SVC(C = 10000).fit(X_Train_add, y_train)
    predictions = clfrNB.predict(X_Test_add)
    
    return roc_auc_score(y_test, predictions)

docl_svc_auc()

0.9490264386221934

### Number of digits

What is the average number of digits per document for not spam and spam documents?

*This function returns a tuple (average # digits not spam, average # digits spam).*

In [12]:
def num_digits():
    nospam = np.mean(spam_data[spam_data['target']==0]['text'].str.count(r'\d'))
    spam = np.mean(spam_data[spam_data['target']==1]['text'].str.count(r'\d'))
    return (nospam, spam)

num_digits()

(0.30515848353014297, 15.812583668005354)

### Adding number of digits

we fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

we fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function returns the AUC score as a float.*

In [13]:
from sklearn.linear_model import LogisticRegression

def numd_logis_auc():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_Train_add = add_feature(X_train_vectorized, [X_train.str.len(),X_train.str.count(r'\d')] )
    X_test_vectorized = vect.transform(X_test)
    X_Test_add = add_feature(X_test_vectorized, [X_test.str.len(), X_test.str.count(r'\d')])
    clfrNB = LogisticRegression(C = 100).fit(X_Train_add, y_train)
    predictions = clfrNB.predict(X_Test_add)
    
    return roc_auc_score(y_test, predictions)

numd_logis_auc()

0.9583489489284862

### Non Alphanumeric Characters

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*This function returns a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [14]:
def non_word():
    nospam = np.mean(spam_data[spam_data['target']==0]['text'].str.count(r'\W'))
    spam = np.mean(spam_data[spam_data['target']==1]['text'].str.count(r'\W'))
    return (nospam, spam)

non_word()

(17.37725295214419, 29.068273092369477)

### Adding non Alpha characters

We fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams we pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This makes the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

we fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **we find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients are sorted smallest first, the list of 10 largest coefficients are sorted largest first.

The three features that were added to the document term matrix have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function returns a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [15]:
def nonal_logistic_auc():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_Train_add = add_feature(X_train_vectorized, [X_train.str.len().rename("length_of_doc"),
                                                   X_train.str.count(r'\d').rename("digit_count"),
                                                   X_train.str.count(r'\W').rename("non_word_char_count")] )
    X_test_vectorized = vect.transform(X_test)
    X_Test_add = add_feature(X_test_vectorized, [X_test.str.len().rename("length_of_doc"), 
                                                 X_test.str.count(r'\d').rename("digit_count"), 
                                                 X_test.str.count(r'\W').rename("non_word_char_count")])
    clfrNB = LogisticRegression(C = 100).fit(X_Train_add, y_train)
    predictions = clfrNB.predict(X_Test_add)
    AUC = roc_auc_score(y_test, predictions)
    
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_tfidf_index = clfrNB.coef_[0].argsort()
    smallest = list(feature_names[sorted_tfidf_index[:10]])
    largest = list(feature_names[sorted_tfidf_index[:-11:-1]])
    
    return (AUC,smallest,largest)

nonal_logistic_auc()

(0.9681356682906886,
 ['..', '? ', ' y', ' i', 'i ', 'h ', ' m', '. ', 'oo', ':)'],
 ['digit_count', 'ww', 'ia', 'xt', 'ne', 'co', ' a ', 'uk', 'mob', ' en'])